##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 学習アルゴリズムを作成する

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/federated/tutorials/composing_learning_algorithms"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">     TensorFlow.org で表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/federated/tutorials/composing_learning_algorithms.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Google Colab で実行</a>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/federated/tutorials/composing_learning_algorithms.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     GitHubでソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/federated/tutorials/composing_learning_algorithms.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

## 始める前に

始める前に、環境が正しくセットアップされていることを確認するために、以下を実行してください。動作しない場合は、[インストール](../install.md)ガイドで手順を確認してください。 

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated

In [ ]:
from collections.abc import Callable

import tensorflow as tf
import tensorflow_federated as tff

**注意**: この Colab は [最新リリースバージョン](https://github.com/tensorflow/federated#compatibility)の `tensorflow_federated` pip パッケージでの動作が確認されていますが、Tensorflow Federated プロジェクトは現在もプレリリース開発の段階にあるため、`main` では動作しない可能性があります。

# 学習アルゴリズムを作成する

「[独自の連合学習アルゴリズムを構築する](https://github.com/tensorflow/federated/blob/v0.62.0/docs/tutorials/building_your_own_federated_learning_algorithm.ipynb)」では、TFF の連合コアを使用して直接 Federated Averaging（FedAvg）アルゴリズムのバージョンを実装しました。

このチュートリアルでは、ゼロからすべてを再実装する必要のないように、TFF の API にある連合学習コンポーネントを使用してモジュール形式で連合学習アルゴリズムをを構築します。

このチュートリアルの目的により、ローカルトレーニングで勾配クリッピングを使用するバリエーションの FedAvg を実装することにします。

## 学習アルゴリズムのビルディングブロック

多数の学習アルゴリズムは、**ビルディングブロック**と呼ばれる以下の 4 つのコンポーネントに大きく分けることができます。

1. ディストリビュータ（サーバーからクライアントへの通信）
2. クライアントワーク（ローカルクライアントの計算）
3. アグリゲータ（クライアントからサーバーへの通信）
4. ファイナライザ（集約したクライアント出力を使用したサーバーの計算）

[「独自の連合学習アルゴリズムを構築する」チュートリアル](https://github.com/tensorflow/federated/blob/v0.62.0/docs/tutorials/building_your_own_federated_learning_algorithm.ipynb)では、これらすべてのビルディングブロックをゼロから実装しましたが、ほとんどの場合、そうする必要はありません。代わりに、似たようなアルゴリズムのビルディングブロックを再利用することができます。

この場合、勾配クリッピングを伴う FedAvg を実装するには、**クライアントワーク**のビルディングブロックのみを変更するだけで済みます。残りのブロックは、「バニラ」FedAvg と同じものを使用することが可能です。

# クライアントワークを実装する

まず、勾配クリッピングでローカルモデルトレーニングを行う TF ロジックを記述しましょう。単純さを考慮し、勾配は最大 1 でノルムを持つようにクリッピングされます。

## TF ロジック

In [ ]:
@tf.function
def client_update(model: tff.learning.models.VariableModel,
                  dataset: tf.data.Dataset,
                  server_weights: tff.learning.models.ModelWeights,
                  client_optimizer: tf.keras.optimizers.Optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = tff.learning.models.ModelWeights.from_model(model)
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  # Keep track of the number of examples as well.
  num_examples = 0.0
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)
      num_examples += tf.cast(outputs.num_examples, tf.float32)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights.trainable)

    # Compute the gradient norm and clip
    gradient_norm = tf.linalg.global_norm(grads)
    if gradient_norm > 1:
      grads = tf.nest.map_structure(lambda x: x/gradient_norm, grads)

    grads_and_vars = zip(grads, client_weights.trainable)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  # Compute the difference between the server weights and the client weights
  client_update = tf.nest.map_structure(tf.subtract,
                                        client_weights.trainable,
                                        server_weights.trainable)

  return tff.learning.templates.ClientResult(
      update=client_update, update_weight=num_examples)

上記のコードについて重要なポイントがいくつかあります。1 つ目は、確認されるサンプル数を追跡することです。これは、クライアント更新の*重み*を構成します（クライアント間の平均を計算する場合）。

2 つ目は、出力のパッケージ化に [`tff.learning.templates.ClientResult`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/templates/ClientResult) を使用していることです。この戻り値の型は、`tff.learning` でクライアントワークのビルディングブロックを標準化するために使用されます。

## ClientWorkProcess を作成する

上記の TF ロジックはクリッピングを伴うローカルトレーニングを実行しますが、必要なビルディングブロックを作成するには、TFF コードでラップされている必要があります。

具体的には、4 つのビルディングブロックは [`tff.templates.MeasuredProcess`](https://www.tensorflow.org/federated/api_docs/python/tff/templates/MeasuredProcess) として表現されます。つまり、4 つすべてのブロックに、計算を初期化して実行するために使用される `initialize` と `next` 関数の両方が含まれるということです。

これにより、各ビルディングブロックは演算を実行するために必要なそれぞれの**状態**（サーバーに保存）を追跡できます。このチュートリアルでは使用されませんが、イテレーションが何回行われたかを追跡したり、オプティマイザの状態を追跡したりすることができます。

クライアントワーク TF ロジックは一般に [`tff.learning.templates.ClientWorkProcess`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/templates/ClientWorkProcess) としてラップされます。これは、クライアントのローカルトレーニングで入出力する期待される型をコード化するものです。モデルとオプティマイザによって、以下のようにパラメータ化することができます。

In [ ]:
def build_gradient_clipping_client_work(
    model_fn: Callable[[], tff.learning.models.VariableModel],
    optimizer_fn: Callable[[], tf.keras.optimizers.Optimizer],
) -> tff.learning.templates.ClientWorkProcess:
  """Creates a client work process that uses gradient clipping."""

  with tf.Graph().as_default():
    # Wrap model construction in a graph to avoid polluting the global context
    # with variables created for this model.
    model = model_fn()
  data_type = tff.SequenceType(model.input_spec)
  model_weights_type = tff.learning.models.weights_type_from_model(model)

  @tff.federated_computation
  def initialize_fn():
    return tff.federated_value((), tff.SERVER)

  @tff.tf_computation(model_weights_type, data_type)
  def client_update_computation(model_weights, dataset):
    model = model_fn()
    optimizer = optimizer_fn()
    return client_update(model, dataset, model_weights, optimizer)

  @tff.federated_computation(
      initialize_fn.type_signature.result,
      tff.type_at_clients(model_weights_type),
      tff.type_at_clients(data_type)
  )
  def next_fn(state, model_weights, client_dataset):
    client_result = tff.federated_map(
        client_update_computation, (model_weights, client_dataset))
    # Return empty measurements, though a more complete algorithm might
    # measure something here.
    measurements = tff.federated_value((), tff.SERVER)
    return tff.templates.MeasuredProcessOutput(state, client_result,
                                               measurements)
  return tff.learning.templates.ClientWorkProcess(
      initialize_fn, next_fn)

# 学習アルゴリズムを作成する

上記のクライアントワークを本格的なアルゴリズムへと展開していきましょう。まず、データとモデルをセットアップします。

## 入力データを準備する

TFF に含まれる EMNIST データセットを読み込んで前処理します。詳細については、[画像分類](federated_learning_for_image_classification.ipynb)チュートリアルをご覧ください。

In [ ]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

データセットをモデルにフィードするには、データをフラット化してタプル形式 `(flattened_image_vector, label)` に変換します。

次に、少数のクライアントを選択し、上記の前処理をデータセットに適用します。

In [ ]:
NUM_CLIENTS = 10
BATCH_SIZE = 20

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape(element['pixels'], [-1, 784]), 
            tf.reshape(element['label'], [-1, 1]))

  return dataset.batch(BATCH_SIZE).map(batch_format_fn)

client_ids = sorted(emnist_train.client_ids)[:NUM_CLIENTS]
federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x))
  for x in client_ids
]

## モデルを準備する

これは、[画像分類](federated_learning_for_image_classification.ipynb)チュートリアルと同じモデルを使用します。このモデル（`tf.keras` 経由で実装）には、非表示レイヤーと、その後にソフトマックスレイヤーが含まれています。このモデルを TFF で使用するために、Keras モデルは [`tff.learning.Model`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model) としてラップします。こうすることで、TFF 内でモデルの[フォワードパス](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#forward_pass)と[モデル出力の抽出](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#report_local_unfinalized_metrics)を実行できるようになります。詳細については、[画像分類](federated_learning_for_image_classification.ipynb)チュートリアルをご覧ください。

In [ ]:
def create_keras_model():
  initializer = tf.keras.initializers.GlorotNormal(seed=0)
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer=initializer),
      tf.keras.layers.Softmax(),
  ])

def model_fn():
  keras_model = create_keras_model()
  return tff.learning.models.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

## オプティマイザを準備する

[`tff.learning.algorithms.build_weighted_fed_avg`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg) と同様に、ここでも、クライアントオプティマイザとサーバーオプティマイザの 2 つがあります。単純さを維持するため、オプティマイザは異なる学習率を伴う SGD とします。

In [ ]:
client_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=0.01)
server_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=1.0)

## ビルディングブロックを定義する

クライアントワークのビルディングブロック、データ、モデル、およびオプティマイザのセットアップが完了したので、後は、ディストリビュータ、アグリゲータ、およびファイナライザのビルディングブロックを作成するのみです。これは、TFF で提供されている、FedAvg で使用されているデフォルトを借りれば完了です。

In [ ]:
@tff.tf_computation()
def initial_model_weights_fn():
  return tff.learning.models.ModelWeights.from_model(model_fn())

model_weights_type = initial_model_weights_fn.type_signature.result

distributor = tff.learning.templates.build_broadcast_process(model_weights_type)
client_work = build_gradient_clipping_client_work(model_fn, client_optimizer_fn)

# TFF aggregators use a factory pattern, which create an aggregator
# based on the output type of the client work. This also uses a float (the number
# of examples) to govern the weight in the average being computed.)
aggregator_factory = tff.aggregators.MeanFactory()
aggregator = aggregator_factory.create(model_weights_type.trainable,
                                       tff.TensorType(tf.float32))
finalizer = tff.learning.templates.build_apply_optimizer_finalizer(
    server_optimizer_fn, model_weights_type)

## ビルディングブロックを作成する

最後に、TFF に組み込みの**コンポーザ**を使用して、ビルディングブロックを 1 つにまとめます。これは比較的単純なコンポーザで、上記の 4 つのビルディングブロックを提供してそれらの型を繋ぎ合わせます。

In [ ]:
fed_avg_with_clipping = tff.learning.templates.compose_learning_process(
    initial_model_weights_fn,
    distributor,
    client_work,
    aggregator,
    finalizer
)

# アルゴリズムを実行する

アルゴリズムが完成したので、実行してみましょう。まず、アルゴリズムを**初期化**します。このアルゴリズムの**状態**には、各ビルディングブロックのコンポーネントと*グローバルモデルの重み*のコンポーネントがあります。

In [ ]:
state = fed_avg_with_clipping.initialize()

state.client_work

()

期待したとおり、クライアントワークの状態は空です（上記のクライアントワークのコードを思い出しましょう！）。ただし、他のビルディングブロックの状態は空以外の場合があります。たとえば、ファイナライザはイテレーションが何回起きたかを追跡しているためです。`next` はまだ実行されていないため、状態は `0` となっています。

In [ ]:
state.finalizer

[0]

では、トレーニングラウンドを実行します。

In [ ]:
learning_process_output = fed_avg_with_clipping.next(state, federated_train_data)

この（`tff.learning.templates.LearningProcessOutput`）出力には、`.state` と `.metrics` の出力があります。両方を確認しましょう。

In [ ]:
learning_process_output.state.finalizer

[1]

明らかに、ファイナライザの状態は、`.next` が実行されたため、1 ずつ増分しています。

In [ ]:
learning_process_output.metrics

OrderedDict([('distributor', ()),
             ('client_work', ()),
             ('aggregator',
              OrderedDict([('mean_value', ()), ('mean_weight', ())])),
             ('finalizer', ())])

メトリックは空ですが、より複雑で実践的なアルゴリズムでは、一般に有用な情報が多数含まれます。

# まとめ

上記のビルディングブロック/コンポーザフレームワークを使用することで、すべてをゼロから作成せずとも、まったく新しい学習アルゴリズムを作成することができます。ただし、これは出発点に過ぎません。このフレームワークによって、アルゴリズムをはるかに簡単に単純な FedAvg の変更コードとして表現できるようになります。詳細については、[`tff.learning.algorithms`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms) をご覧ください。これには、[FedProx](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_prox) や[クライアント学習率のスケジューリングを伴う FedAvg](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg_with_optimizer_schedule) などのアルゴリズムが含まれています。これらの API を使うと、[連合 k-平均クラスタリング](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_fed_kmeans)など、まったく新しいアルゴリズムの実装の支援をさらに得られます。